# Actualizar base de conocimiento del chatbot

Este archivo permite actualizar la base de conocimiento la cual es usada como fuente de información para el chabot


Link del chatbot: https://www.dev.galgo.com/app/chat-motos-mx/chat

In [8]:
import sys
sys.path.append('../')  # subir 1 nivel desde notebooks a src
from google import genai
from src.config.settings import STORE_ID, GEMINI_API_KEY, COUNTRY, KNOWLEDGE_BASE_STORE_FILENAME, LAST_DOCUMENT_UPLOADED

In [9]:
print("🔧 Iniciando cliente...")
client = genai.Client(api_key=GEMINI_API_KEY)

🔧 Iniciando cliente...


## Actualizar store

[link a la documentación de Stores](https://ai.google.dev/api/file-search/file-search-stores?hl=es-419#method:-filesearchstores.get)

https://www.philschmid.de/gemini-file-search-javascript

### Actualizar documento de un store
Para actualizar un documento de un Store, se tiene que eliminar el documento anterior y subir el nuevo. Se puede leer [acá](https://www.philschmid.de/gemini-file-search-javascript)

In [10]:
import time

In [11]:
def delete_execive_documents_from_store(lista_documentos:list):
    """
    Elimina documentos exceso de documentos de un store. Solamente debería haber uno.
    """
    lista_documentos_a_eliminar = lista_documentos[1:len(lista_documentos)] # Se deja el primero que es el mas reciente, el resto se elimina
    for document in lista_documentos_a_eliminar:
        print("Eliminando documento: ", document)
        client.file_search_stores.documents.delete(name = document, config = {"force": True})
        print(f"Documento eliminado: {document}")
    print("Fin de eliminación de documentos")

In [12]:
def delete_document_from_store(document):
    client.file_search_stores.documents.delete(name = document, config = {"force": True})
    print(f"Documento eliminado: {document}")

In [13]:
def get_documents_from_store(store_id:str):
    documents = client.file_search_stores.documents.list(parent = store_id)
    if not documents:
        raise Exception("No hay documentos en el store")
    return documents

## Actualizar base de conocimiento

1. Consulta los archivos del Store
2. Valida que solamente haya un archivo, en caso que haya mas de uno, elimina todos dejando solamente el último subido
3. Elimina el archivo existente
4. Sube el nuevo archivo
5. Lista nuevamente los archivos (solamente debe haber uno)

In [14]:
documents = get_documents_from_store(STORE_ID)
# Itera sobre los documentos y guarda sus nombres
lista_documentos = []
for d in documents:
    print(d)
    lista_documentos.append(d.name)

name='fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-29czmymhwt04' display_name='base_conocimiento_chatbot.md' state=<DocumentState.STATE_ACTIVE: 'STATE_ACTIVE'> size_bytes=20421 mime_type='text/plain' create_time=datetime.datetime(2026, 1, 23, 21, 55, 15, 5862, tzinfo=TzInfo(0)) custom_metadata=None update_time=datetime.datetime(2026, 1, 23, 21, 55, 16, 42197, tzinfo=TzInfo(0))


In [15]:
import os
from datetime import datetime

def log_update_event(message):
    """ Seguimiento de cada vez que se actualice el Store """
    log_dir = "../src/data/output/logs"
    os.makedirs(log_dir, exist_ok=True)
    log_path = os.path.join(log_dir, "knowledge_base_update_log.txt")
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_path, "a", encoding="utf-8") as log_file:
        log_file.write(f"[{now}] {message}\n")

# Lista documentos del Store
documents = get_documents_from_store(STORE_ID)
# Itera sobre los documentos y guarda sus nombres
lista_documentos = []
for d in documents:
    print(d)
    lista_documentos.append(d.name)

if len(lista_documentos) > 1:
    print("Hay más de 1 documento en el store, eliminando los demás")
    delete_execive_documents_from_store(lista_documentos)
    log_update_event(f"Eliminados {len(lista_documentos)-1} documentos extra del store {STORE_ID}")
else:
    print ("Solo hay un documento en el store, reemplazando documento por uno nuevo (se va a eliminar el existente y subir el nuevo)")
    # Eliminar el documento existente
    delete_document_from_store(lista_documentos[0])
    log_update_event(f"Eliminado documento anterior {lista_documentos[0]} del store {STORE_ID}")
    # Subir nuevo documento
    print("Abriendo archivo nuevo...")
    file_path = f"../src/data/output/gemini/{COUNTRY}-knowledge_base_final.md"
    with open(file_path, 'rb') as contenido:
        print("Subiendo archivo al Store...")
        operation = client.file_search_stores.upload_to_file_search_store(
            file=contenido,
            file_search_store_name=STORE_ID,
            config={
                'mime_type': 'text/plain',
                'display_name': KNOWLEDGE_BASE_STORE_FILENAME
            }
        )
        print(f"   ✅ Archivo subido e indexado correctamente")
        log_update_event(f"Subido nuevo archivo {os.path.basename(file_path)} al store {STORE_ID}")

documents = get_documents_from_store(STORE_ID)
# Itera sobre los documentos y guarda sus nombres
lista_documentos = []
for d in documents:
    print(d)
    lista_documentos.append(d.name)

name='fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-29czmymhwt04' display_name='base_conocimiento_chatbot.md' state=<DocumentState.STATE_ACTIVE: 'STATE_ACTIVE'> size_bytes=20421 mime_type='text/plain' create_time=datetime.datetime(2026, 1, 23, 21, 55, 15, 5862, tzinfo=TzInfo(0)) custom_metadata=None update_time=datetime.datetime(2026, 1, 23, 21, 55, 16, 42197, tzinfo=TzInfo(0))
Solo hay un documento en el store, reemplazando documento por uno nuevo (se va a eliminar el existente y subir el nuevo)
Documento eliminado: fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-29czmymhwt04
Abriendo archivo nuevo...
Subiendo archivo al Store...
   ✅ Archivo subido e indexado correctamente
name='fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-hx5rok0i9p87' display_name='base_conocimiento_chatbot.md' state=<DocumentState.STATE_PENDING: 'STATE_PENDING'> size_bytes=918

Consultar los documentos en el Store

In [16]:
documents = get_documents_from_store(STORE_ID)
# Itera sobre los documentos y guarda sus nombres
lista_documentos = []
for d in documents:
    print(d)
    lista_documentos.append(d.name)

name='fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-hx5rok0i9p87' display_name='base_conocimiento_chatbot.md' state=<DocumentState.STATE_ACTIVE: 'STATE_ACTIVE'> size_bytes=91889 mime_type='text/plain' create_time=datetime.datetime(2026, 1, 27, 16, 51, 39, 981579, tzinfo=TzInfo(0)) custom_metadata=None update_time=datetime.datetime(2026, 1, 27, 16, 51, 41, 616603, tzinfo=TzInfo(0))


In [ ]:
# # Lista documentos del Store
# documents = client.file_search_stores.documents.list(parent = STORE_ID)
# if not documents:
#     print("No hay documentos en el store")
# # Iterar entre cada documento del store
# lista_documentos = [] # Solo debería de haber uno, por lo tanto, si hya mas de 1, se eliminan
# for d in documents:
#     print(d)
#     # Si hay mas de 1, se eliminan los demás
#     if len(lista_documentos) > 1:
#         lista_documentos.append(d.name)
#         lista_documentos_a_eliminar = lista_documentos[1:len(lista_documentos)] # Se deja el primero que es el mas reciente, el resto se elimina
#         for document in lista_documentos_a_eliminar:
#             client.file_search_stores.documents.delete(name = f"fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/{document}", config = {"force": True})

#    # Verificar nuevamente los archivos del store
#     documents = client.file_search_stores.documents.list(parent = STORE_ID)
#     if not documents:
#         print("No hay documentos en el store")
#     for d in documents:
#         print(d)

name='fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-29czmymhwt04' display_name='base_conocimiento_chatbot.md' state=<DocumentState.STATE_ACTIVE: 'STATE_ACTIVE'> size_bytes=20421 mime_type='text/plain' create_time=datetime.datetime(2026, 1, 23, 21, 55, 15, 5862, tzinfo=TzInfo(0)) custom_metadata=None update_time=datetime.datetime(2026, 1, 23, 21, 55, 16, 42197, tzinfo=TzInfo(0))
name='fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-rnh9vd7x8kll' display_name='base_conocimiento_chatbot.md' state=<DocumentState.STATE_ACTIVE: 'STATE_ACTIVE'> size_bytes=20421 mime_type='text/plain' create_time=datetime.datetime(2026, 1, 23, 21, 56, 2, 104275, tzinfo=TzInfo(0)) custom_metadata=None update_time=datetime.datetime(2026, 1, 23, 21, 56, 3, 242959, tzinfo=TzInfo(0))
name='fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-y5m85n1wxibg' display_name='base_conocimient

3

['fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-rnh9vd7x8kll',
 'fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-y5m85n1wxibg']

In [ ]:
# # Eliminar un documento específico
# client.file_search_stores.documents.delete(name = "fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-n6p28k11530a",
# config = {"force": True})

In [8]:
# Actualizar documento de un store
# Paso 1: Eliminar el documento anterior (asegurarse de tener solo uno para evitar que haya mezcla de información)

# Verificar los documentos del Store
documents = client.file_search_stores.documents.list(parent = STORE_ID)
if not documents:
    print("No hay documentos en el store")
# Iterar entre cada documento del store
for d in documents:
    # No hace falta el IF, fue solo de pruebas. Debería haber un solo documento en el store para que no haya mezcla de información
    if d.name == "fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-0f1cpkw8a81j":
        client.file_search_stores.documents.delete(name = d.name, config = {"force": True}) # Eliminar el único documento existente del store
# Abrir el archivo nuevo
with open(f"../src/data/output/gemini/{COUNTRY}-knowledge_base_final.md", 'rb') as contenido:
    operation = client.file_search_stores.upload_to_file_search_store(
        file=contenido,
        file_search_store_name=STORE_ID,
        config={
            'mime_type': 'text/plain',
            'display_name': KNOWLEDGE_BASE_STORE_FILENAME
        }
    )

    # # Esperar a que se suba el archivo, sino, reintenta en 2 segundos
    # while not operation.done:
    #     time.sleep(2)
    #     operation = client.operations.get(name=operation.name)

    # # En caso de error, lanzar excepción
    # if operation.error:
    #     raise Exception(f"Error: {operation.error}")
    print(f"   ✅ Archivo subido e indexado correctamente")

print("Verificando archivos subidos al store...")
documents = client.file_search_stores.documents.list(parent = STORE_ID)
if not documents:
    print("No hay documentos en el store")
# Iterar entre cada documento del store
for d in documents:
    print(d)

   ✅ Archivo subido e indexado correctamente
Verificando archivos subidos al store...
name='fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-29czmymhwt04' display_name='base_conocimiento_chatbot.md' state=<DocumentState.STATE_ACTIVE: 'STATE_ACTIVE'> size_bytes=20421 mime_type='text/plain' create_time=datetime.datetime(2026, 1, 23, 21, 55, 15, 5862, tzinfo=TzInfo(0)) custom_metadata=None update_time=datetime.datetime(2026, 1, 23, 21, 55, 16, 42197, tzinfo=TzInfo(0))
name='fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-rnh9vd7x8kll' display_name='base_conocimiento_chatbot.md' state=<DocumentState.STATE_ACTIVE: 'STATE_ACTIVE'> size_bytes=20421 mime_type='text/plain' create_time=datetime.datetime(2026, 1, 23, 21, 56, 2, 104275, tzinfo=TzInfo(0)) custom_metadata=None update_time=datetime.datetime(2026, 1, 23, 21, 56, 3, 242959, tzinfo=TzInfo(0))
name='fileSearchStores/galgo-base-conocimiento-v2-90uyev7

In [ ]:
import time
# ==========================================
# 1️⃣ LIMPIAR ARCHIVOS ANTERIORES
# ==========================================

print(f"\n1️⃣  Limpiando archivos anteriores del store...")
try:
    existing_files = list(client.file_search_stores.list_files(
        file_search_store_name=STORE_ID
    ))

    if len(existing_files) > 0:
        for file_info in existing_files:
            print(f"   🗑️  Eliminando: {file_info.display_name}")
            client.file_search_stores.delete_file(
                file_search_store_name=STORE_ID,
                file_name=file_info.name
            )
        print(f"   ✅ {len(existing_files)} archivo(s) eliminado(s)")
    else:
        print(f"   ℹ️  No había archivos previos")

except Exception as e:
    print(f"   ⚠️  Advertencia al limpiar: {e}")
    print(f"   ℹ️  Continuando...")

# ==========================================
# 2️⃣ SUBIR NUEVO ARCHIVO AL STORE
# ==========================================

print(f"\n2️⃣  Subiendo archivo al File Search Store...")
try:
    # Upload directo al store (indexación automática)
    with open(f"../src/data/output/gemini/{COUNTRY}-knowledge_base_final.md", 'rb') as contenido:
        operation = client.file_search_stores.upload_to_file_search_store(
            file=contenido,
            file_search_store_name=STORE_ID,
            config={
                'mime_type': 'text/plain',
                'display_name': KNOWLEDGE_BASE_STORE_FILENAME
            }
        )

    print(f"   ⏳ Esperando procesamiento e indexación vectorial...")

    # Esperar a que termine
    while not operation.done:
        time.sleep(2)
        operation = client.operations.get(name=operation.name)

    # Verificar resultado
    if operation.error:
        raise Exception(f"Error: {operation.error}")

    print(f"   ✅ Archivo subido e indexado correctamente")

except Exception as e:
    print(f"   ❌ Error: {e}")
    raise

🔧 Reparando subida (Versión Corregida) para Store: fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass
1️⃣  Sube tu archivo .md (Markdown) original:


Saving base_conocimiento_hero_hunk125R_colombia_.md to base_conocimiento_hero_hunk125R_colombia_.md
   ✅ Recibido localmente: base_conocimiento_hero_hunk125R_colombia_.md
   📝 Renombrado internamente a: base_conocimiento_chatbot.md

2️⃣  Subiendo a la nube de Google (Forzando text/plain)...
   ✅ Archivo subido exitosamente. ID: files/w4ar6j48bnob
   ⏳ Esperando procesamiento de Google...
   ✅ Estado del archivo: FileState.ACTIVE

3️⃣  Vinculando archivo files/w4ar6j48bnob al Store fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass...

✅ ¡ÉXITO TOTAL! El archivo se subió y vinculó.
Ahora ejecuta de nuevo el script de AUDITORÍA y debería aparecer.


## Verificar archivos en el store

In [27]:
info_list = client.file_search_stores.list()
info_list

In [11]:
info

FileSearchStore(
  active_documents_count=4,
  create_time=datetime.datetime(2026, 1, 16, 15, 31, 27, 588753, tzinfo=TzInfo(0)),
  display_name='Galgo - Base Conocimiento V2',
  name='fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass',
  size_bytes=62100,
  update_time=datetime.datetime(2026, 1, 16, 15, 31, 27, 588753, tzinfo=TzInfo(0))
)

In [6]:
print(f"🔍 AUDITORÍA DE ARCHIVOS EN PROYECTO")
print(f"🎯 Buscando archivo: {KNOWLEDGE_BASE_STORE_FILENAME}")
print("=" * 60)

try:
    # 1. Listar archivos (SIN parámetros extra que causan error)
    # La librería maneja la paginación automáticamente si iteramos
    all_files = client.files.list()

    found = False
    count = 0

    print("📂 Escaneando lista de archivos en la nube...")

    for f in all_files:
        # Filtramos para no llenar la pantalla de basura, solo lo que nos importa
        # O mostramos todo si hay pocos archivos

        # Verificamos si es nuestro archivo
        is_target = False
        if hasattr(f, 'display_name') and (KNOWLEDGE_BASE_STORE_FILENAME in f.display_name or "conversaciones" in f.display_name):
            is_target = True
            found = True

        # Imprimimos solo si es el objetivo O si queremos ver los últimos 5 para depurar
        if is_target or count < 5:
            state_icon = "✅" if f.state == "ACTIVE" else "⏳" if f.state == "PROCESSING" else "❌"

            print(f"\n📄 Archivo encontrado:")
            print(f"   • Nombre: {f.display_name}")
            print(f"   • Estado: {state_icon} {f.state}")
            print(f"   • ID:     {f.name}")
            print(f"   • Creado: {f.create_time}")

            if is_target:
                print("   👉 ¡ESTE ES EL QUE BUSCAMOS!")

        count += 1

    print("\n" + "=" * 60)
    if found:
        print("✅ CONCLUSIÓN: El archivo SÍ está en la nube de Google.")
        print("Si Lovable no responde, es 100% seguro que Lovable necesita un 'Redeploy'.")
    else:
        print("❌ CONCLUSIÓN: El archivo NO aparece en la lista.")
        print("Hubo un error silencioso en la subida. Debemos intentar subirlo de nuevo.")

except Exception as e:
    print(f"❌ Error al listar: {e}")

🔍 AUDITORÍA DE ARCHIVOS EN PROYECTO
🎯 Buscando archivo: base_conocimiento_chatbot.md
📂 Escaneando lista de archivos en la nube...

📄 Archivo encontrado:
   • Nombre: base_conocimiento_chatbot.md
   • Estado: ✅ FileState.ACTIVE
   • ID:     files/597soxkf4mde
   • Creado: 2026-01-23 20:49:33.073424+00:00
   👉 ¡ESTE ES EL QUE BUSCAMOS!

✅ CONCLUSIÓN: El archivo SÍ está en la nube de Google.
Si Lovable no responde, es 100% seguro que Lovable necesita un 'Redeploy'.


In [ ]:
# # Verificar los documentos activos en el Store
# nombre_documentos_en_store = []
# info = client.file_search_stores.documents.list(parent = STORE_ID)
# for f in info:
#     # print(f) # Ver el objeto
#     nombre_documentos_en_store.append(f.name)


# # Eliminar un documento por nombre
# try:
#     client.file_search_stores.documents.delete(name='fileSearchStores/galgo-base-conocimiento-v2-90uyev7f3ass/documents/baseconocimientochatbotmd-n6p28k11530a',
#                                                 config = {"force": True})
#     print("Archivo eliminado")
#     info = client.file_search_stores.documents.list(parent = STORE_ID)
#     for f in info:
#         print(f"Información disponible del STORE")
#         print(f)
# except Exception as e:
#     print(f"Error al eliminar el archivo: {e}")

# Subir archivo a un FileSearchStore
# operation = client.file_search_stores.upload_to_file_search_store(
#     file=CONTENIDO_ARCHIVO,
#     file_search_store_name=STORE_ID,
#     config={
#         'mime_type': 'text/plain',
#         'display_name': KNOWLEDGE_BASE_STORE_FILENAME
#     }
# )